In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from scipy.linalg import sqrtm

In [3]:
DATA_PATH = 'dataset/ml-100k-dataset'
MOVIE_FILENAME = 'u.item'
RATING_FILENAME = 'u.data'

movie_filename_path = os.path.join(DATA_PATH, MOVIE_FILENAME)
rating_filename_path = os.path.join(DATA_PATH, RATING_FILENAME)

In [4]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(rating_filename_path, sep='\t', names=r_cols,
                      encoding='latin-1')

In [14]:
print(ratings.head(5).to_string(index=False))
print(ratings.shape)

 user_id  movie_id  rating  unix_timestamp
     196       242       3       881250949
     186       302       3       891717742
      22       377       1       878887116
     244        51       2       880606923
     166       346       1       886397596
(100000, 4)


In [6]:
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv(movie_filename_path, sep='|', names=m_cols, usecols=range(5),
                     encoding='latin-1')

In [7]:
def get_test_train_split(df, test_ratio):
    users = df['user_id'].unique()
    movies = df['movie_id'].unique()
    
    test = None
    train = None

    for u in users:
        temp = df[df['user_id'] == u]
        # n = len(temp)
        temp_train, temp_test = train_test_split(temp, test_size=0.2)
        # print(temp_test.shape, temp_train.shape)
        if test is not None:
            test = pd.concat([test, temp_test])
            train = pd.concat([train, temp_train])
        else:
            test = temp_test
            train = temp_train
    return test, train

In [8]:
ratings_test, ratings_train = get_test_train_split(ratings, 0.2)

In [9]:
train, test = train_test_split(ratings, test_size=0.2)
uim = train.pivot(index='user_id', columns='movie_id', values='rating')

In [15]:
user_item_matrix = ratings_train.pivot(index='user_id', columns='movie_id', values='rating')
user_item_matrix.head(20)
print(user_item_matrix.shape)

(943, 1646)


In [11]:
def svd(train, k):
    util_mat = np.array(train)
    mask = np.isnan(util_mat)
    masked_arr = np.ma.masked_array(util_mat, mask)
    item_means = np.mean(masked_arr, axis=0)
    util_mat = masked_arr.filled(item_means)
    x = np.tile(item_means, (util_mat.shape[0], 1))
    util_mat = util_mat - x

    U,s,V = np.linalg.svd(util_mat, full_matrices=False)
    
    s=np.diag(s)
    s=s[0:k,0:k]
    U=U[:,0:k]
    V=V[0:k,:]
    
    s_root = sqrtm(s)

    Usk = np.dot(U, s_root)
    skV = np.dot(s_root, V)
    UsV = np.dot(Usk, skV)

    UsV = UsV + x
    return UsV

In [12]:
def rmse(true, pred):
    X = true - pred
    return sum([xi*xi for xi in X])/len(X)

In [16]:
THE_CHOOSEN_USER = 196

In [26]:
import time

no_of_features_decomposition = [5,10,15,20,25]


for f in no_of_features_decomposition:
    print("--------------------------------------------------------------------")
    start_time = time.time()
    print("1. Building model starting for " + str(f) + " SVD Projection Dimension ")
    svdout = svd(user_item_matrix, f)
    print("2. Built model for " + str(f) + " SVD Projection Dimension in " + str(time.time() - start_time) + "s")

    pred = []

    print("3. Predictions start...")
    start_time = time.time()
    for _,row in ratings_test.iterrows():
        user = row['user_id']
        item = row['movie_id']
        pred_rating = None
        if item <= len(user_item_matrix.columns):
            pred_rating = svdout[user-1, item-1]
        else:
            pred_rating = np.mean(svdout[user-1,:])
        pred.append(pred_rating)
    print("4. Predictions complete in " + str(time.time() - start_time) + "s for " + str(len(pred)) + " data-points")

    results_df = ratings_test.copy(deep=True)
    results_df.drop(columns=['unix_timestamp'], inplace=True)
    results_df['pred_rating'] = pred
    users = results_df['user_id'].unique()
    results_df.rename(columns={"rating": "actual_rating"}, inplace=True)
    results_df_str = None
    for user in users:
        temp = results_df[results_df['user_id'] == user]
        if results_df_str is None:
            results_df_str = temp.head(1)
        else:
            results_df_str = pd.concat([results_df_str, temp.head(1)])
    
    choosen_user_preds = svdout[THE_CHOOSEN_USER-1, :]
    best_movies = np.flip(np.argsort(choosen_user_preds))
    print("Top 5 movies recommended for the choosen UserID 196 with predicted ratings")
    counter = 1
    for movie in best_movies:
        if counter > 5:
            break
        if ratings[(ratings['movie_id'] == movie+1) & (ratings['user_id'] == THE_CHOOSEN_USER)].shape[0] < 1:
            print(str(counter) + ". " + movies[movies['movie_id'] == movie+1].iloc[0]['title'] + "; Predicted Rating - " + str(choosen_user_preds[movie]))
            counter += 1
    
    print("5. Predictions Output (For 20 unique users)")
    print(results_df_str.head(20).to_string(index=False))

    print("6. RMSE for " + str(f) + " SVD Projection Dimension = " + str(rmse(ratings_test['rating'], pred)))

--------------------------------------------------------------------
1. Building model starting for 5 SVD Projection Dimension 
2. Built model for 5 SVD Projection Dimension in 0.6738145351409912s
3. Predictions start...
4. Predictions complete in 0.7941782474517822s for 20381 data-points
Top 5 movies recommended for the choosen UserID 196 with predicted ratings
1. That Old Feeling (1997); Predicted Rating - 5.0
2. Price Above Rubies, A (1998); Predicted Rating - 5.0
3. Umbrellas of Cherbourg, The (Parapluies de Cherbourg, Les) (1964); Predicted Rating - 5.0
4. Young Guns II (1990); Predicted Rating - 5.0
5. Damsel in Distress, A (1937); Predicted Rating - 5.0
5. Predictions Output (For 20 unique users)
 user_id  movie_id  actual_rating  pred_rating
     196      1118              4     3.698713
     186       939              5     2.390029
      22        29              1     2.500097
     244      1095              2     2.502977
     166       894              4     3.258764
     

In [24]:
ratings[ratings['user_id'] == THE_CHOOSEN_USER]

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
940,196,393,4,881251863
1133,196,381,4,881251728
1812,196,251,3,881251274
1896,196,655,5,881251793
2374,196,67,5,881252017
6910,196,306,4,881251021
7517,196,238,4,881251820
7842,196,663,5,881251911
10017,196,111,4,881251793
